In [1]:
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
# Realizamos una petición al servicio rest y comprobamos que se ha realizado correctamente (código de estado = 200)
request = requests.get("https://sedeaplicaciones.minetur.gob.es/ServiciosRESTCarburantes/PreciosCarburantes/EstacionesTerrestres/")
print(request.status_code)

200


In [3]:
# Formateamos la respuesta en formato JSON
json_data = request.json()

In [4]:
# Almacenamos por separado los valores correspondientes a la fecha de la petición y al listado de los precios de las estaciones de servicio
fecha = json_data["Fecha"].split(" ")[0].replace("/","-")
precios = json_data["ListaEESSPrecio"]
print(fecha)

08-04-2022


In [5]:
# Creamos un dataframe con los precios y nos quedamos únicamente con las columnas relativas a los precios del combustible
precios_df = pd.json_normalize(precios)

precios_df = precios_df[["IDEESS", "Precio Biodiesel", "Precio Bioetanol", "Precio Gas Natural Comprimido", "Precio Gas Natural Licuado", "Precio Gases licuados del petróleo", "Precio Gasoleo A", "Precio Gasoleo B", "Precio Gasoleo Premium", "Precio Gasolina 95 E10", "Precio Gasolina 95 E5", "Precio Gasolina 95 E5 Premium", "Precio Gasolina 98 E10", "Precio Gasolina 98 E5", "% BioEtanol", "% Éster metílico"]]

precios_df = precios_df.replace("", np.NaN) # Sustituimos los valores vacíos por NaN

print(precios_df.shape)
precios_df.head()

(11654, 16)


,IDEESS,Precio Biodiesel,Precio Bioetanol,Precio Gas Natural Comprimido,Precio Gas Natural Licuado,Precio Gases licuados del petróleo,Precio Gasoleo A,Precio Gasoleo B,Precio Gasoleo Premium,Precio Gasolina 95 E10,Precio Gasolina 95 E5,Precio Gasolina 95 E5 Premium,Precio Gasolina 98 E10,Precio Gasolina 98 E5,% BioEtanol,% Éster metílico
0,4375,NaN,NaN,NaN,NaN,NaN,"1,829","1,340",NaN,NaN,"1,799",NaN,NaN,NaN,"0,0","0,0"
1,5122,NaN,NaN,NaN,NaN,NaN,"1,950",NaN,"2,000",NaN,"1,869",NaN,NaN,"2,000","0,0","0,0"
2,12054,NaN,NaN,NaN,NaN,NaN,"1,859",NaN,"1,949",NaN,"1,799",NaN,NaN,"1,941","0,0","0,0"
3,10765,NaN,NaN,NaN,NaN,NaN,"1,859",NaN,"1,889",NaN,"1,799",NaN,NaN,"1,899","0,0","0,0"
4,4438,NaN,NaN,NaN,NaN,NaN,"1,879",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"


In [6]:
# Comprobamos los valores nulos de cada columna por si existe alguna cuyo todos los registros sean nulos
precios_df.isna().sum(axis = 0)

IDEESS                                    0
Precio Biodiesel                      11616
Precio Bioetanol                      11649
Precio Gas Natural Comprimido         11543
Precio Gas Natural Licuado            11578
Precio Gases licuados del petróleo    10846
Precio Gasoleo A                        328
Precio Gasoleo B                       8636
Precio Gasoleo Premium                 4107
Precio Gasolina 95 E10                11636
Precio Gasolina 95 E5                  1049
Precio Gasolina 95 E5 Premium         10922
Precio Gasolina 98 E10                11648
Precio Gasolina 98 E5                  5595
% BioEtanol                               0
% Éster metílico                          0
dtype: int64

In [7]:
# Insertamos una columna correspondiente a la fecha para poder distinguir entre los precios en distintos días
precios_df.insert(0, "Fecha", fecha)  

In [8]:
# Comprobamos el resultado de las operaciones realizadas
precios_df.head()

,Fecha,IDEESS,Precio Biodiesel,Precio Bioetanol,Precio Gas Natural Comprimido,Precio Gas Natural Licuado,Precio Gases licuados del petróleo,Precio Gasoleo A,Precio Gasoleo B,Precio Gasoleo Premium,Precio Gasolina 95 E10,Precio Gasolina 95 E5,Precio Gasolina 95 E5 Premium,Precio Gasolina 98 E10,Precio Gasolina 98 E5,% BioEtanol,% Éster metílico
0,08-04-2022,4375,NaN,NaN,NaN,NaN,NaN,"1,829","1,340",NaN,NaN,"1,799",NaN,NaN,NaN,"0,0","0,0"
1,08-04-2022,5122,NaN,NaN,NaN,NaN,NaN,"1,950",NaN,"2,000",NaN,"1,869",NaN,NaN,"2,000","0,0","0,0"
2,08-04-2022,12054,NaN,NaN,NaN,NaN,NaN,"1,859",NaN,"1,949",NaN,"1,799",NaN,NaN,"1,941","0,0","0,0"
3,08-04-2022,10765,NaN,NaN,NaN,NaN,NaN,"1,859",NaN,"1,889",NaN,"1,799",NaN,NaN,"1,899","0,0","0,0"
4,08-04-2022,4438,NaN,NaN,NaN,NaN,NaN,"1,879",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,0","0,0"


In [9]:
# Almacenamos el resultado en un fichero con el formato precioEESS-mes-año.csv
precios_df.to_csv(f"../data/preciosEESS-{fecha[3:]}.csv", sep=";", header=True, index=False, encoding="utf-8")